# 

In [ ]:
# !pip install datasets transformers torchaudio jiwer

In [23]:
from datasets import load_dataset
import os
import json

audio_dir = "./creolese-audio-dataset/Audio Files"
transcript_file = "./creolese-audio-dataset/transcripts.json"

with open(transcript_file, "r", encoding="utf-8") as f:
    transcript_data = json.load(f)  # this gives you a list of dicts

# Now we extract paths and texts
audio_files = [entry["audio"] for entry in transcript_data]
transcripts = [entry["text"] for entry in transcript_data]

print(f"Found {len(audio_files)} audio files and {len(transcripts)} transcripts")

Found file: ./creolese-audio-dataset/Audio Files/1.wav
Found file: ./creolese-audio-dataset/Audio Files/2.wav
Found file: ./creolese-audio-dataset/Audio Files/3.wav
Found file: ./creolese-audio-dataset/Audio Files/4.wav
Found file: ./creolese-audio-dataset/Audio Files/5.wav
Found file: ./creolese-audio-dataset/Audio Files/6.wav
Found file: ./creolese-audio-dataset/Audio Files/7.wav
Found file: ./creolese-audio-dataset/Audio Files/8.wav
Found file: ./creolese-audio-dataset/Audio Files/9.wav
Found file: ./creolese-audio-dataset/Audio Files/10.wav
Found file: ./creolese-audio-dataset/Audio Files/11.wav
Found file: ./creolese-audio-dataset/Audio Files/12.wav
Found file: ./creolese-audio-dataset/Audio Files/13.wav
Found file: ./creolese-audio-dataset/Audio Files/14.wav
Found file: ./creolese-audio-dataset/Audio Files/15.wav
Found file: ./creolese-audio-dataset/Audio Files/16.wav
Found file: ./creolese-audio-dataset/Audio Files/17.wav
Found file: ./creolese-audio-dataset/Audio Files/18.wav
F

In [26]:
dataset = Dataset.from_list(data)

# Cast the audio column to automatically load audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
print(dataset)



Dataset({
    features: ['audio', 'text'],
    num_rows: 36
})


In [20]:
split_dataset = dataset.train_test_split(test_size=0.2)


In [56]:
from transformers import Wav2Vec2Processor

# Load processor (tokenizer + feature extractor)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

# def prepare_dataset(batch):
#     audio = batch["audio"]

#     # Extract input features
#     batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

#     # Encode labels (transcription)
#     batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    

#     return batch

def prepare_dataset(batch):
    audio = batch["audio"]

    # Get input values from audio
    input_values = processor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt"
    ).input_values[0]

    # Get labels from text
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    

    # Return proper format for CTC
    return {
        "input_values": input_values,
        "labels": batch["labels"]
    }

# Apply preprocessing
processed_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names, num_proc=4)


Map (num_proc=4):   0%|          | 0/36 [00:00<?, ? examples/s]

In [57]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id
)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
import transformers
print(transformers.__version__)

4.51.3


# This is a custom attempt

In [63]:
import torch
from dataclasses import dataclass
from typing import Dict, List, Union, Any

@dataclass
class SimpleCTCDataCollator:
        processor: Any

        def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
            # Get the input_values from each feature
            input_values = [feature["input_values"].squeeze(0) if isinstance(feature["input_values"], torch.Tensor) else torch.tensor(feature["input_values"]) for feature in features]

            # Determine max length for padding
            max_length = max(len(x) for x in input_values)

            # Pad the input_values manually
            padded_input_values = []
            attention_mask = []

            for val in input_values:
                # Create attention mask (1 for real values, 0 for padding)
                length = len(val)
                mask = torch.ones(length)
                if length < max_length:
                    pad_length = max_length - length
                    # Pad the input values
                    val = torch.nn.functional.pad(val, (0, pad_length), value=0.0)
                    # Extend the attention mask with zeros for padding
                    mask = torch.nn.functional.pad(mask, (0, pad_length), value=0.0)

                padded_input_values.append(val)
                attention_mask.append(mask)

            # Stack the padded inputs and attention masks
            batch = {
                "input_values": torch.stack(padded_input_values),
                "attention_mask": torch.stack(attention_mask)
            }

            # Get labels
            if "labels" in features[0]:
                labels = [feature["labels"] for feature in features]

                # Pad labels manually with -100 (ignore index for CTC loss)
                padded_labels = []
                max_label_length = max(len(l) for l in labels)

                for label in labels:
                    if isinstance(label, torch.Tensor):
                        label = label.tolist()

                    if len(label) < max_label_length:
                        # Pad with -100
                        label = label + [-100] * (max_label_length - len(label))

                    padded_labels.append(torch.tensor(label, dtype=torch.long))

                batch["labels"] = torch.stack(padded_labels)

            return batch

In [64]:
data_collator = SimpleCTCDataCollator(processor=processor)

# This is the version that doesn't work

In [58]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=processor.tokenizer, padding=True)


In [59]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./wav2vec2-creolese-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=25,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    fp16=False,  # True if on GPU with mixed precision
    gradient_accumulation_steps=2
)


In [60]:
import jiwer
import torch

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = torch.argmax(torch.tensor(pred_logits), dim=-1)
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = jiwer.wer(label_str, pred_str)
    mer = jiwer.mer(label_str, pred_str)
    cer = jiwer.cer(label_str, pred_str)
    return {"wer": wer, "mer": mer, "cer": cer}
    

In [61]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset,
    eval_dataset=processed_dataset,
    tokenizer=processor.feature_extractor
)


/tmp/ipykernel_96203/3842381790.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [65]:
trainer.train()
trainer.evaluate()

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['labels', 'input_values']

In [ ]:
model.save_pretrained("./wav2vec2-creolese-finetuned")
processor.save_pretrained("./wav2vec2-creolese-finetuned")
